<a href="https://colab.research.google.com/github/paulynamagana/AFDB_notebooks/blob/main/AFDB_BigQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Accessing AlphaFold DB structures through BigQuery**

### Introduction
<img src="https://www.vectorlogo.zone/logos/google_bigquery/google_bigquery-icon.svg" height="200" align="right">

The metadata for all ~214 million predicted structure models from AlphaFold can be accessed with BigQuery.

**IMPORTANT: The free tier of Google Cloud comes with BigQuery Sandbox with 1 TB of free processed query data each month. Repeated queries within a month could exceed this limit and if you have upgraded to a paid Cloud Billing account you may be charged.**

*This should be sufficient for running a number of queries on the metadata table, though the usage depends on the size of the columns queried and selected. Please look at the BigQuery pricing page for more information. This is the user's responsibility so please ensure you keep track of your billing settings and resource usage in the console.*

**IMPORTANT:** <br>
Before we get started, it's important to note that you will need to carefully follow the provided instructions that will help you set up the required environment and configure the necceasary dependencies. Make sure you have access to a Google account, as we will be using Google Colab and BigQuery.


<br>

In order to customise the code you will need to **Save a copy** of this Notebook to your Drive. You can visit this resource [Query Syntax](https://cloud.google.com/bigquery/docs/reference/standard-sql/query-syntax#select_list) for further information about SQL queries in GoogleSQL for BigQuery.





---
## Before you start, set up BigQuery <a name="Set up Big Query"></a>

Before you start running any code below, follow the BigQuery Sandbox [Set up guide](https://cloud.google.com/bigquery/docs/sandbox). The link also discusses options to upgrade if the limited capabilities are not enough for your work. Moreover, it highlights the limitations of this sandbox.

To set up a BigQuery Sandbox and get started with accessing AlphaFold DB structures, follow these instructions:

1. Log in to your Google Account, if you don't have a Google account, create one by clicking on "Create account" and following the on-screen instructions.
2. Go to the [BigQuery page](https://console.cloud.google.com/bigquery)
. Start using the sandbox by creating a project name of your choice (follow instructions in the link above). </br>
  *Important Note: When setting up your BigQuery Sandbox, please be aware that it does not accept names with spaces or underscores. Make sure to choose names that consist of alphanumeric characters only and separate words with dashes.*
3. Copy your `project-id` to use in the cell to Authenticate and create a client to access BigQuery.




---



## How to use Google Colab <a name="How to use GoogleColab"></a>
1. To run a code cell, click on the cell to select it. You will notice a play button (▶️) on the left side of the cell. Click on the play button or press Shift+Enter to **run** the code in the selected cell.
2. The code will start executing, and you will see the output, if any, displayed below the code cell.
3. Move to the next code cell and repeat steps 2 and 3 until you have executed all the desired code cells in sequence.
4. The currently running step is indicated by a circle with a stop sign next to it.
If you need to stop or interrupt the execution of a code cell, you can click on the stop button (■) located next to the play button.

*Remember to run the code cells in the correct order, as their execution might depend on variables or functions defined in previous cells. You can modify the code in a code cell and re-run it to see updated results.*



In [ ]:
#@title #Change `project_id` and run this cell
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table
data_table.enable_dataframe_formatter()
import pandas as pd


# Authenticate and create a client to access BigQuery
auth.authenticate_user()
project_id = "serious-energy-400810" #@param {type:"string"}
#@markdown `project_id` is not the same as project name. To see the `project_id` visit your [dashboard](https://console.cloud.google.com/cloud-resource-manager?)

client = bigquery.Client(project=project_id)


def execute_bigquery_and_create_dataframe(query):
    """
    Execute a BigQuery query, parse the results, and create a pandas DataFrame.

    Parameters:
    - query (str): The SQL query to be executed.
    - client: The BigQuery client.

    Returns:
    - pd.DataFrame: A pandas DataFrame containing the query results.
    """
    # Execute the query and retrieve the results using BigQuery client
    client = bigquery.Client(project=project_id)
    query_job = client.query(query)
    results = query_job.result()

    # Parse the results and store them as a list of dictionaries
    data = [dict(row) for row in results]

    # Create a pandas DataFrame to view the results as a table
    df = pd.DataFrame(data)

    return df


## 1.1.&nbsp;  Data Types
The code blocks below query the column names and their respective data types of the entire table. This ensures the column names are up-to-date and also facilitates a clear view of the metadata table's structure, for subseqeunt new queries. Moreover, this also tests if your setup is working correctly.

In [ ]:
# This SQL query retrieves the column names and corresponding data types for the 'metadata' table
query = """
SELECT column_name, data_type

FROM bigquery-public-data.deepmind_alphafold.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'metadata'
"""

#Big Query
execute_bigquery_and_create_dataframe(query)

,column_name,data_type
0,allVersions,ARRAY<INT64>
1,latestVersion,INT64
2,organismCommonNames,ARRAY<STRING>
3,uniprotEnd,INT64
4,proteinShortNames,ARRAY<STRING>
5,uniprotStart,INT64
6,fractionPlddtConfident,FLOAT64
7,organismSynonyms,ARRAY<STRING>
8,fractionPlddtVeryHigh,FLOAT64
9,proteinFullNames,ARRAY<STRING>


You can read the description of each column, observed below in your query.

|**Column name**	| **Data type** |	**Description**|
---------|-----------|--------------------- |
|allVersions|	ARRAY <INT64>|	An array of AFDB versions this prediction has had
|entryId	| STRING	| The AFDB entry ID, e.g. "AF-Q1HGU3-F1"|
|fractionPlddtConfident	|FLOAT64	|Fraction of the residues in the prediction with pLDDT between 70 and 90|
|fractionPlddtLow	|FLOAT64	|Fraction of the residues in the prediction with pLDDT between 50 and 70|
|fractionPlddtVeryHigh	|FLOAT64	|Fraction of the residues in the prediction with pLDDT greater than 90|
|fractionPlddtVeryLow	|FLOAT64	|Fraction of the residues in the prediction with pLDDT less than 50|
|globalMetricValue	|FLOAT64	|The mean pLDDT of this prediction|
|isReferenceProteome	|BOOL	|Is this protein part of the reference proteome?
|isReviewed	|BOOL|	Has this protein been reviewed, i.e. is it part of SwissProt?|
|latestVersion	|INT64|	The latest AFDB version for this prediction
|modelCreatedDate	|DATE	|The date of creation for this entry, e.g. "2022-06-01"
|organismCommonNames	|ARRAY<STRING>|	List of common organism names
|organismScientificName	|STRING|	The scientific name of the organism
|organismSynonyms	|ARRAY<STRING>	|List of synonyms for the organism
|proteinFullNames	|ARRAY<STRING>	|Full names of the protein
|proteinShortNames	|ARRAY<STRING>|	Short names of the protein
|sequenceChecksum	|STRING	|CRC64 hash of the sequence. Can be used for cheaper lookups.
|sequenceVersionDate	|DATE	|Date when the sequence data was last modified in UniProt

<br>

For more detailed information, check the [GitHub repository](https://github.com/deepmind/alphafold/blob/main/afdb/README.md#exploring-the-metadata)



## 1.2.&nbsp;  Basic Query
Now, we can start querying for specific entries to work with, depending on the data you require for downstream purposes. To explore the data, this next query will return the first 10 entries to view the data for them.

In [ ]:
## SQL query to retrieve a limited set of data from the AlphaFold metadata table
query = """
SELECT *

FROM `bigquery-public-data.deepmind_alphafold.metadata` --DATASET
LIMIT 10 --LIMIT THE QUERY
"""

execute_bigquery_and_create_dataframe(query)

,allVersions,latestVersion,organismCommonNames,uniprotEnd,proteinShortNames,uniprotStart,fractionPlddtConfident,organismSynonyms,fractionPlddtVeryHigh,proteinFullNames,...,uniprotId,modelCreatedDate,fractionPlddtVeryLow,sequenceVersionDate,entryId,geneSynonyms,uniprotSequence,gene,isReferenceProteome,isReviewed
0,"[2, 3, 4]",4,[Bovine],151,[],1,0.053,[],0.927,[],...,NGB_BOVIN,2022-06-01,0.007,2006-06-13,AF-Q6WZ19-F1,[],MELPEPELIRQSWREVSRSPLEHGTVLFARLFDLEPDLLPLFQYNC...,NGB,True,True
1,"[3, 4]",4,[Crab-eating macaque],89,[],1,0.112,[Cynomolgus monkey],0.865,[],...,Q1JUY0_MACFA,2022-06-01,0.000,2006-06-13,AF-Q1JUY0-F1,[],ENYLYQGRQECYAINGTQRFLERHIYNREEFMRFDSDVGEYRAVTE...,Mafa-DPB1,False,False
2,"[3, 4]",4,[],218,[],1,0.050,[],0.950,[],...,Q1HJ49_9BACT,2022-06-01,0.000,2006-06-13,AF-Q1HJ49-F1,[],YVGQEKLRSQSGWLPLAFALDWSRPPRQQNSTSFFYAHTDQWRYET...,narG,False,False
3,"[3, 4]",4,[],198,[],1,0.616,[],0.025,[],...,Q1HD62_9CREN,2022-06-01,0.126,2006-06-13,AF-Q1HD62-F1,[],CTHYLFIVVVAVNSTLLTINTGDYIFYTDWAWTSYTVFSIGNILMI...,amoA,False,False
4,"[3, 4]",4,[],857,[],1,0.244,[],0.603,[],...,Q1I458_PSEE4,2022-06-01,0.117,2006-06-13,AF-Q1I458-F1,[],MADWQSLDPEAAREAEKYDNPIPSRELILQRLADRGEPAAREQLAE...,rnr,True,False
5,"[3, 4]",4,[],42,[],1,0.405,[],0.000,[],...,Q1IAW0_PSEE4,2022-06-01,0.000,2006-06-13,AF-Q1IAW0-F1,[],MDMLTNLQTHTLKAEAEAEAEAEAEAEAKAKIKAPPAPSPET,PSEEN2398,True,False
6,"[3, 4]",4,[],181,[],1,0.337,[],0.481,[],...,Q1IEH0_PSEE4,2022-06-01,0.039,2006-06-13,AF-Q1IEH0-F1,[],MRTTIVRYIKDRSFLATLTSVTSDYPGNLDSVEGMPHKRGNAWLTV...,PSEEN1033,True,False
7,"[3, 4]",4,[],310,[],1,0.277,[],0.623,[],...,Q1IAX1_PSEE4,2022-06-01,0.003,2006-06-13,AF-Q1IAX1-F1,[],MDMLHAMRTFARVVECGSFAAAANALDISSAQVSRIVAELENQLQT...,PSEEN2387,True,False
8,"[2, 3, 4]",4,[],295,[],1,0.349,[],0.542,[Heat shock protein HtpX],...,HTPX_PSEE4,2022-06-01,0.078,2006-06-13,AF-Q1ID18-F1,[],MMRILLFVATNLAVVLVASITLSLFGFNGFMAANGVDLNLGQLLVF...,htpX,True,True
9,"[3, 4]",4,[],207,[],1,0.343,[],0.565,[],...,Q1IFC2_PSEE4,2022-06-01,0.039,2006-06-13,AF-Q1IFC2-F1,[],MLASLTAFVITSAARLITGARALWLGCTPQPVQRLYYANHSSHGDF...,PSEEN0706,True,False


##1.3 Advanced Query



###1.3.1&nbsp; Retrieve prediction confidence fractions per species.

However, this query has been limited to the top 31 species according to the num_predictions as it  may take time to retrieve the full grouped dataset.

In [ ]:
query= """SELECT
 organismScientificName AS name, --HAVE THE SCIENTIFIC NAME AS NAME
  ROUND(SUM(fractionPlddtVeryLow) / COUNT(fractionPlddtVeryLow), 3) AS mean_plddt_very_low,
  ROUND(SUM(fractionPlddtLow) / COUNT(fractionPlddtLow), 3) AS mean_plddt_low,
  ROUND(SUM(fractionPlddtConfident) / COUNT(fractionPlddtConfident), 3) AS mean_plddt_confident,
  ROUND(SUM(fractionPlddtVeryHigh) / COUNT(fractionPlddtVeryHigh), 3) AS mean_plddt_very_high,
  COUNT(organismScientificName) AS num_predictions

FROM bigquery-public-data.deepmind_alphafold.metadata --DATASET
GROUP BY name --GROUP BY ORGANISM NAME
ORDER BY num_predictions DESC -- SORT RESULTS
LIMIT 31; -- LIMITED """

execute_bigquery_and_create_dataframe(query)

,name,mean_plddt_very_low,mean_plddt_low,mean_plddt_confident,mean_plddt_very_high,num_predictions
0,Acidobacteria bacterium,0.090,0.100,0.220,0.590,1066443
1,Chloroflexi bacterium,0.075,0.088,0.211,0.627,1018842
2,Actinobacteria bacterium,0.062,0.083,0.205,0.650,924571
3,Gammaproteobacteria bacterium,0.063,0.082,0.211,0.644,889963
4,Escherichia coli,0.079,0.123,0.249,0.549,864366
5,Deltaproteobacteria bacterium,0.085,0.094,0.223,0.598,809761
6,marine sediment metagenome,0.138,0.181,0.249,0.432,666660
7,Mesorhizobium sp,0.056,0.094,0.203,0.647,664158
8,Helicobacter pylori,0.076,0.099,0.233,0.592,552054
9,Bacteroidetes bacterium,0.065,0.083,0.222,0.630,485548


###1.3.2&nbsp; Retrieve proteins from a specific gene name for all organisms

In this case we're retrieving only the information from the next columns:

* organismScientificName
* globalMetricValue
* uniprotId
* modelCreatedDate
* latestVersion
* sequenceVersionDate
* uniprotSequence

You can modify the query accordingly, look for the comment `-- COLUMNS TO RETRIEVE`

In [ ]:
query= """SELECT organismScientificName, globalMetricValue, uniprotId, modelCreatedDate, latestVersion, sequenceVersionDate, uniprotSequence --COLUMNS TO RETRIEVE
  FROM bigquery-public-data.deepmind_alphafold.metadata --DATASET

  WHERE gene = "SLC22A1" -- GENE TO SEARCH
  ORDER BY globalMetricValue --SORT RESULTS"""

execute_bigquery_and_create_dataframe(query)

,organismScientificName,globalMetricValue,uniprotId,modelCreatedDate,latestVersion,sequenceVersionDate,uniprotSequence
0,Symbiodinium sp. CCMP2592,44.59,A0A812HJV6_9DINO,2022-06-01,4,2021-09-29,MQVGEVGFLALAAAAFLVSTVCVLPLQETHCTELKDFVEPHEGESR...
1,Eumeta variegata,53.34,A0A4C1ZRY2_EUMVA,2022-06-01,4,2019-07-03,MFQPCDLAEDRLLHCAHLASVNNIVSHYLTLSFDSASDLYYKVGFD...
2,Symbiodinium microadriaticum,53.81,A0A1Q9CJ80_SYMMI,2022-06-01,4,2017-04-12,MPGDAQPGLSESDSRGSHQEAHAAHRRSGPLLRRRGSQMTLQTTTK...
3,Symbiodinium sp. KB8,57.00,A0A813B9B9_9DINO,2022-06-01,4,2021-09-29,MTLQTTTKTDGATGLFTVTAARLPELIDEIGMGMFQIIQFLPVLFV...
4,Ursus maritimus,64.62,A0A452U5X1_URSMA,2022-06-01,4,2019-05-08,MPTVDDILEQVGEFGRFQKQTFLALCLLSAAFAPIYVGIVFLAFTP...
...,...,...,...,...,...,...,...
119,Chlorocebus sabaeus,84.31,A0A0D9RIX5_CHLSB,2022-06-01,4,2015-05-27,MPTVDDILEQVGEFGWFQKQAFLTLCLLSAAFAPICVGVVFLGFTP...
120,Bos taurus,84.43,S22A1_BOVIN,2022-06-01,4,2007-10-02,MLTVDDVLEQVGEFGWFQKQTFLILCLLSAAFAPIYVGIVFLAFTP...
121,Homo sapiens,84.96,S22A1_HUMAN,2022-06-01,4,2010-11-30,MPTVDDILEQVGESGWFQKQAFLILCLLSAAFAPICVGIVFLGFTP...
122,Oryctolagus cuniculus,85.86,S22A1_RABIT,2022-06-01,4,1998-11-01,MPTVDDVLEQVGEFGWFQKRTFLFLCLISAILAPIYLGIVFLGFTP...


### 1.3.3&nbsp;  Create a subset with pLDDT <70 for an Organism
The next query involves creating subsets of proteins according to various criteria. For example, the next query below only includes proteins from *Homo sapiens* for which over half the residues are confident or better (>70 pLDDT).

In [ ]:
# Define the query with LIMIT clause
#YOU CAN MODIFY THE QUERY ACCORDING TO YOUR NEEDS
query = """
    SELECT entryId,  modelCreatedDate, uniprotId, uniprotDescription, sequenceVersionDate
    FROM bigquery-public-data.deepmind_alphafold.metadata -- DATASET

    WHERE organismScientificName = "Homo sapiens" -- ORGANISM TO SEARCH
      AND (fractionPlddtVeryHigh + fractionPlddtConfident) > 0.5 -- FILTER BY PLDDT
    LIMIT 20  -- LIMIT TO 10 QUERIES
    -----
"""

## Where fractionPlddtVeryHigh == Fraction of the residues in the prediction with pLDDT greater than 90
##fractionPlddtConfident ==	Fraction of the residues in the prediction with pLDDT between 70 and 90

execute_bigquery_and_create_dataframe(query)

,entryId,modelCreatedDate,uniprotId,uniprotDescription,sequenceVersionDate
0,AF-I7B2B2-F1,2022-06-01,I7B2B2_HUMAN,MHC class I antigen,2012-10-03
1,AF-J3QQQ0-F1,2022-06-01,J3QQQ0_HUMAN,Phospholipase A1,2012-10-03
2,AF-A7MAV4-F1,2022-06-01,A7MAV4_HUMAN,UBE2Q1 protein,2007-11-13
3,AF-A0A0X7YLE9-F1,2022-06-01,A0A0X7YLE9_HUMAN,MHC class II antigen,2016-04-13
4,AF-Q3T7C7-F1,2022-06-01,Q3T7C7_HUMAN,tRNA isopentenylpyrophosphate transferase isof...,2005-10-11
5,AF-V9HWF3-F1,2022-06-01,V9HWF3_HUMAN,Epididymis luminal protein 74,2014-03-19
6,AF-V9J548-F1,2022-06-01,V9J548_HUMAN,NADH-ubiquinone oxidoreductase chain 4,2014-03-19
7,AF-A8K8B9-F1,2022-06-01,A8K8B9_HUMAN,cAMP-dependent protein kinase,2007-12-04
8,AF-H0YFD1-F1,2022-06-01,H0YFD1_HUMAN,"28S ribosomal protein S35, mitochondrial",2012-02-22
9,AF-H0YFQ1-F1,2022-06-01,H0YFQ1_HUMAN,Transmembrane protein 132A,2012-02-22


###1.3.4&nbsp;   Prepare query for download

You can customise the next query to download the specific data from a query that you have tuned for your specific requirements.

Look for the comment `CHANGE THE NEXT LINES OF THE QUERY` and paste your query, make sure you **do not** include the `SELECT` statement.


In [ ]:
# Define the query with LIMIT clause
#YOU CAN MODIFY THE QUERY ACCORDING TO YOUR NEEDS
query = """
with file_rows AS (
  with file_cols AS (
    SELECT
      CONCAT(entryID, '-model_v4.cif') as m,
      CONCAT(entryID, '-predicted_aligned_error_v4.json') as p
    FROM bigquery-public-data.deepmind_alphafold.metadata

    ---- CHANGE THE NEXT LINES OF THE QUERY TO YOUR SPECIFIC QUERY
    WHERE organismScientificName = "Homo sapiens" -- ORGANISM TO SEARCH
      AND (fractionPlddtVeryHigh + fractionPlddtConfident) > 0.5 -- FILTER BY PLDDT
    LIMIT 10  -- Limit the query to the first 10 entries
    -----

  )
  SELECT * FROM file_cols UNPIVOT (files for filetype in (m, p))
)
SELECT CONCAT('gs://public-datasets-deepmind-alphafold-v4/', files) as files
from file_rows
"""

## Where fractionPlddtVeryHigh == Fraction of the residues in the prediction with pLDDT greater than 90
##fractionPlddtConfident ==	Fraction of the residues in the prediction with pLDDT between 70 and 90

# Execute the query
query_job = client.query(query)

# Fetch the query results
results = query_job.result()

# Create a list to store the results
file_list = []
# Iterate over the results and extract the "files" column value
for row in results:
    file_list.append(row["files"])

for entry in file_list:
  print(entry)

In [ ]:
#@title Save files to Google Drive
#@markdown **Note:** this block of code will download all the files listed from the previous code to your Google Drive into a folder called "AFDB_BigQuery".
import os
from google.colab import drive

drive.mount('/content/drive')
destination_path = '/content/drive/MyDrive/AFDB_BigQuery'

# Check whether the specified path exists or not
isExist = os.path.exists(destination_path)
if not isExist:
    # Create a new directory because it does not exist
    os.makedirs(destination_path)
    print("The new directory is created!")

# Function to download a file from Google Cloud Storage and save it to the destination
def download_gs_file(gs_url, dest_path):
    # Extract the filename from the GS URL
    filename = os.path.basename(gs_url)

    # Use gsutil to copy the file from GS to the destination path
    copy_command = f"gsutil cp {gs_url} {os.path.join(dest_path, filename)}"

    # Execute the gsutil command
    os.system(copy_command)
    print("Downloaded:", filename)

# Run this code to download the structures from Google Cloud Storage
for structure in file_list:
    # Use the download function to copy files from GS to the destination
    download_gs_file(structure, destination_path)


## Contact us

If you experience any bugs please contact afdbhelp@ebi.ac.uk




---

